In [5]:
# read file
with open('../samples/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [6]:
# basic stats
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


In [7]:
# character encoder/decoder
atoi = { ch: i for i, ch in enumerate(chars) }
itoa = { i: ch for i, ch in enumerate(chars) }
encode = lambda s: [atoi[ch] for ch in s]
decode = lambda l: ''.join([itoa[i] for i in l])

print(encode(text[:100]))
print(decode(encode(text[:100])))

[18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 14, 43, 44, 53, 56, 43, 1, 61, 43, 1, 54, 56, 53, 41, 43, 43, 42, 1, 39, 52, 63, 1, 44, 59, 56, 58, 46, 43, 56, 6, 1, 46, 43, 39, 56, 1, 51, 43, 1, 57, 54, 43, 39, 49, 8, 0, 0, 13, 50, 50, 10, 0, 31, 54, 43, 39, 49, 6, 1, 57, 54, 43, 39, 49, 8, 0, 0, 18, 47, 56, 57, 58, 1, 15, 47, 58, 47, 64, 43, 52, 10, 0, 37, 53, 59]
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [8]:
# encode the entire dataset
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [11]:
# separate the dataset
n = int(0.9 * len(data))
train_data = data[:n]
validation_data = data[n:]

In [12]:
# example of a batch of training samples
sequence_length = 8
x = train_data[:sequence_length]
y = train_data[1:sequence_length + 1]
for t in range(sequence_length):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [31]:
# example of a batch
batch_size = 4
sequence_length = 8  # the maximum length for prediction

def get_batch(split):
    data = train_data if split == 'train' else validation_data
    # random sampling subsequence starting positions from the dataset
    sampled_pos = torch.randint(len(data) - sequence_length, size=(batch_size, ))
    x = torch.stack([data[i : i + sequence_length] for i in sampled_pos])
    y = torch.stack([data[i + 1 : i + sequence_length + 1] for i in sampled_pos])
    return x, y

x_batch, y_batch = get_batch('train')
print('inputs:')
print(x_batch.shape)
print(x_batch)
print('targets:')
print(y_batch.shape)
print(y_batch)

print('------')
for ib in range(batch_size):
    for it in range(sequence_length):
        context = x_batch[ib, :it + 1]
        target = y_batch[ib, it]
        print(f"input: {context}, target: {target}")

inputs:
torch.Size([4, 8])
tensor([[57, 41, 53, 60, 43, 56, 63,  1],
        [53, 49, 43, 52, 10,  0, 37, 53],
        [56, 43, 51, 47, 58, 47, 43, 57],
        [41, 46,  1, 42, 39, 56, 49, 43]])
targets:
torch.Size([4, 8])
tensor([[41, 53, 60, 43, 56, 63,  1, 50],
        [49, 43, 52, 10,  0, 37, 53, 59],
        [43, 51, 47, 58, 47, 43, 57,  1],
        [46,  1, 42, 39, 56, 49, 43, 52]])
------
input: tensor([57]), target: 41
input: tensor([57, 41]), target: 53
input: tensor([57, 41, 53]), target: 60
input: tensor([57, 41, 53, 60]), target: 43
input: tensor([57, 41, 53, 60, 43]), target: 56
input: tensor([57, 41, 53, 60, 43, 56]), target: 63
input: tensor([57, 41, 53, 60, 43, 56, 63]), target: 1
input: tensor([57, 41, 53, 60, 43, 56, 63,  1]), target: 50
input: tensor([53]), target: 49
input: tensor([53, 49]), target: 43
input: tensor([53, 49, 43]), target: 52
input: tensor([53, 49, 43, 52]), target: 10
input: tensor([53, 49, 43, 52, 10]), target: 0
input: tensor([53, 49, 43, 52, 10,

In [32]:
# a simple bigram LM
import torch
import torch.nn as nn
from torch.nn import functional as F

class BigramLM(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn .Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both with size (B, T)
        logits = self.token_embedding_table(idx)  # (B, T, C), where C = vocab_size

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # reshape as cross_entropy receive channel as the second dimension.
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        """Generate the next character."""
        # idx is (B, T) array of indices in the current context.
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # only need the last in the sequence even the entire sequence is available. This is how the bigram model works.
            logits = logits[:, -1, :]  # (B, T, C) -> (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T + 1)
        return idx
    

     

blm = BigramLM(vocab_size)
out, loss = blm(x_batch, y_batch)
print(out.shape)
print(loss)

# The input sequence represented by idx.
idx = torch.zeros((1, 1), dtype=torch.long)
# generated sequence with length 100
generated_sequence = blm.generate(idx, max_new_tokens=100)
print(f"generated_sequence shape: {generated_sequence.shape}")
print(f'generated sequence: {decode(generated_sequence[0].tolist())}')


torch.Size([32, 65])
tensor(4.7504, grad_fn=<NllLossBackward0>)
generated_sequence shape: torch.Size([1, 101])
generated sequence: 
rExM,,
a!zDkA3V.
yqVp&a.a'FVXdGJC n&SvYIqzbSHVa
Haa-,of
MVa3o;HRBQg33333jzA?pv.Y?SCUJd;YYp3Py,WffFfk


In [33]:
# optimizer
optimizer = torch.optim.AdamW(blm.parameters(), lr=1e-3)

In [37]:
# train
batch_size = 32
for step in range(10000):

    # sample a batch of data
    x_batch, y_batch = get_batch('train')

    # evaluate the train loss
    logits, loss = blm(x_batch, y_batch)
    optimizer.zero_grad(set_to_none=True)  # reset the grad of the previous step
    loss.backward()  # calculate the new grad
    optimizer.step()  # perform weight update
    if step % 100 == 0:
        print(f'{step}: loss: {loss.item()}')

0: loss: 2.387354850769043
100: loss: 2.3983991146087646
200: loss: 2.5503296852111816
300: loss: 2.451212167739868
400: loss: 2.4221627712249756
500: loss: 2.412201404571533
600: loss: 2.4494588375091553
700: loss: 2.4233133792877197
800: loss: 2.576375722885132
900: loss: 2.5179200172424316
1000: loss: 2.439497709274292
1100: loss: 2.30306339263916
1200: loss: 2.6035003662109375
1300: loss: 2.532226800918579
1400: loss: 2.4742531776428223
1500: loss: 2.4114925861358643
1600: loss: 2.5322859287261963
1700: loss: 2.479283094406128
1800: loss: 2.477593183517456
1900: loss: 2.6023712158203125
2000: loss: 2.302628517150879
2100: loss: 2.5529701709747314
2200: loss: 2.390684127807617
2300: loss: 2.5232696533203125
2400: loss: 2.486433744430542
2500: loss: 2.456812620162964
2600: loss: 2.416308641433716
2700: loss: 2.4526450634002686
2800: loss: 2.447007417678833
2900: loss: 2.437631130218506
3000: loss: 2.5642058849334717
3100: loss: 2.4085869789123535
3200: loss: 2.4282164573669434
3300: 

In [40]:
# reevaluate the output
# The input sequence represented by idx.
idx = torch.zeros((1, 1), dtype=torch.long)
# generated sequence with length 100
generated_sequence = blm.generate(idx, max_new_tokens=300)
print(f'generated sequence: {decode(generated_sequence[0].tolist())}')

generated sequence: 
Bulo fangond ithof ngr, thiserircrd at'su coularus hale
As eay abu LAndsadonen:
Be, brth, sod sateespond u,
I
An ya ghananceingre.
T:
Buir us hare
Tomoransme gee se.
TZAUCEDutheadup. n, mangoknd far, ngn the I My RO:

An wirr:
Wacanatharegrdouth drore ss,
NCEd gnseds ouriee Peten.
O t s whe t;
S:

T
